## Setting Up the Environment

🎯 Create and set up the workspace, experiment, and environment associated with this project.


In [1]:
from azureml.core import Workspace, Experiment, Environment

ws = Workspace.get(name="quick-starts-ws-147214", resource_group = "aml-quickstarts-147214", subscription_id = "81cefad3-d2c9-4f77-a466-99a7f541c7bb")

exp = Experiment(workspace=ws, name="bank-offer-success-prediction")

env = Environment.get(workspace=ws, name="AzureML-Tutorial")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-147214
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-147214


## Setting Up the AzureML Compute Target Cluster
🎯 Retrieve or create an ML compute target cluster to be used for training. If creating, use the Standard_D2_V2 template, which consists of 4 cores CPU and 7 GB of RAM. 

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException

cluster_name = "cpu-cluster-1"
try:
    cluster = ComputeTarget(workspace = ws, name = cluster_name)
    print('Found existing cluster with specified name. Using it!')
except ComputeTargetException:
    print('Did not found existing cluster with specified name. Creating it!')
    config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cluster = ComputeTarget.create(ws, cluster_name, config)
    
cluster.wait_for_completion(show_output=True)

Did not found existing cluster with specified name. Creating it!
Creating.........
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Preparing HyperDrive Run

🎯Attempt to find the best hyperparameters (*C, max_iter*) for a logistic regression run, using the following specifications.
* Sampling strategy - Random sampling, for both hyperparameters.
* Early stopping policy - Bandit policy, that early stops runs when their current accuracy is more than 0.2 worse than the best accuracy of the run. 
* Run configuration - The start script of the training process, the compute target cluster and the environment

In [3]:
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, randint
import os

sampler = RandomParameterSampling(parameter_space = {"C": uniform(1, 10), 
                                                "max_iter": randint(3000)})

policy = BanditPolicy(evaluation_interval = 1, slack_factor = 0.2, delay_evaluation = 5)


run_configuration = ScriptRunConfig(source_directory = '.',
                                    script = "train.py",
                                    compute_target = cluster,
                                    environment = env)

hyperdrive_config = HyperDriveConfig(run_config = run_configuration,
                                     hyperparameter_sampling = sampler,
                                     policy = policy,
                                     primary_metric_name = "accuracy",
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

## Submit and Visualize the HyperDrive Run

🎯Start the automatic hyperparameter tuning process and display the live results of the runs.

In [4]:
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Preparing and Preprocessing Data

🎯 Prepare and clean the data for further processing. 

In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory
from train import clean_data, split_variables

train_csv_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
test_csv_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv"

train_ds = TabularDatasetFactory.from_delimited_files(train_csv_path)
test_ds = TabularDatasetFactory.from_delimited_files(test_csv_path)

train_ds = clean_data(train_ds)
test_ds = clean_data(test_ds)

x_train, y_train = split_variables(train_ds)
x_test, y_test = split_variables(test_ds)

x_train.head(5)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
0,57,1,0,0,1,5,1,371,1,999,...,1,0,0,0,0,1,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,0,1,0,0,0,0,0,0,0,1
2,33,1,0,0,0,5,5,52,1,999,...,1,0,0,0,1,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,0,1,0,0,0,1,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,1,0,0,0,0,1,0,0,0,0


## Getting the Best HyperDrive Model

🎯 Find the model with the highest accuracy from the hyperparameter tuning process. Train the model on the train data, then save it.

In [6]:
import joblib
from sklearn.linear_model import LogisticRegression

highest_accuracy_run = hyperdrive_run.get_best_run_by_primary_metric()
highest_accuracy_run

Experiment,Id,Type,Status,Details Page,Docs Page
bank-offer-success-prediction,HD_b9b42c8b-b1cd-47fd-b0cb-1b0e4cd4a815_12,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [7]:
highest_accuracy_log_reg_model = LogisticRegression(C=3.67, max_iter=1700).fit(x_train, y_train)
joblib.dump(highest_accuracy_log_reg_model, './outputs/log-reg-highest-acc-model.pkl')

['./outputs/log-reg-highest-acc-model.pkl']

## Configuring the AutoML Run 
🎯 Configure the AutoML parameters.

* Aim to maximize the accuracy
* Use 5-fold cross-validation

In [9]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "experiment_timeout_minutes" : 30,
    "enable_early_stopping": True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    "primary_metric": "accuracy",
    "featurization": "auto"
}

automl_config = AutoMLConfig(task='classification',
                             training_data=train_ds,
                             label_column_name='y',
                             n_cross_validations=5,
                             **automl_settings)

In [10]:
automl_run = exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
bank-offer-success-prediction,AutoML_f99d7fbe-4ab9-414c-adb4-2aac7849e5ab,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias toward

## Getting the Best AutoML Model

🎯 Get the model with the highest accuracy.

In [11]:
import joblib

highest_accuracy_automl_run, highest_accuracy_automl_model = automl_run.get_output()

print(highest_accuracy_automl_model)

joblib.dump(highest_accuracy_automl_model, './outputs/automl-highest-acc-model.pkl')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
    gpu_training_param_dict={'processing_unit_type': 'cpu'}
), random_state=None, reg_alpha=0.5789473684210527, reg_lambda=0.631578947368421, subsample=0.6436842105263159))], verbose=False))], flatten_transform=None, weights=[0.25, 0.3333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333]))],
         verbose=False)


['./outputs/automl-highest-acc-model.pkl']

## Evaluating on Test Data

🎯 Evaluate both the hyperdrive model and the automl model on the test data.

* Random Sampler Tuned Logistic Regression: **0.9135** Test Accuracy
* Gradient-boosted-tree-based Voting Ensemble: **0.9168** Test Accuracy

In [12]:
log_reg_test_accuracy = highest_accuracy_log_reg_model.score(x_test, y_test)
log_reg_test_accuracy

0.9094660194174757

In [13]:
automl_test_accuracy = highest_accuracy_automl_model.score(x_test, y_test)
automl_test_accuracy

0.9174757281553398

## Cleaning the Compute
🎯 Delete the AML Compute instance

In [14]:
try:
    cluster.delete()
    print('Successfully deleted the allocated compute cluster.')
except ComputeTargetException:
    print('The compute AML cluster was not created via Azure Machine Learning, therefore it cannot be deleted programtically. Nothing has happened.')

Successfully deleted the allocated compute cluster.
